<a href="https://colab.research.google.com/github/y-arjun-y/liars-miller-rabin/blob/main/liars_miller_rabin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Liars in the Miller-Rabin Primality Test by Arjun Yadav
## All code is available on [GitHub](https://github.com/y-arjun-y/liars-miller-rabin).

## Implementation of the Miller-Rabin primality test in Python

In [66]:
'''
Function by https://rosettacode.org/wiki/Miller%E2%80%93Rabin_primality_test#Python with slight changes to support a custom witness and explanation of the code. 
Function and its formula is based on https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test.
A return value of False means n in not prime (i.e. composite).
A return value of True means n is very likely to be a prime (depending on the number of rounds).
'''

# base function
def miller_rabin_base(n, a):
    # checking if it is an integer
    if n != int(n):
        return False

    n = int(n)

    # base cases
    if n == 0 or n == 1 or n == 4 or n == 6 or n == 8 or n == 9:
        return False

    if n == 2 or n == 3 or n == 5 or n == 7:
        return True

    if n % 2 == 0:
        return False

    # assigning variables
    s = 0
    d = n - 1

    while d % 2 == 0:
        d >>= 1
        s += 1
    
    assert(2**s * d == n - 1)

    # trial run
    def trial_composite(a):
        if pow(a, d, n) == 1:
            return False

        for i in range(s):
            if pow(a, 2**i * d, n) == n - 1:
                return False
        return True

    # number of trials, directly proportional to time and accuracy.
    num_trials = 10

    # true run
    for _ in range(num_trials):
        if trial_composite(a):
            return False

    return True

True

## Supporting multiple witnesses in the Miller-Rabin primality test

In [88]:
'''
Returns False if any one witness returns False and True if all witnesses return True.
Based on the following facts about the Miller-Rabin primality test:
  A return value of False means n in not prime (i.e. composite).
  A return value of True means n is very likely to be a prime (depending on the number of rounds).
'''

def miller_rabin(n, witness_array):
  res = []
  
  for i in range(len(witness_array)):
    res.append(miller_rabin_base(n, witness_array[i]))
  
  if False in res:
    return False
  
  return True

## Length of probable primes function (using the Miller-Rabin primality test)

In [2]:
'''
Returns the length of prime numbers of a given range.
'''

def len_probable_prime(int, a):
    results = []

    # main loop
    for i in range(int + 1):
        if miller_rabin(i, a) == True:
            results.append(miller_rabin(i, a))

    return len(results)

## Actual number of primes between two numbers



In [3]:
'''
Returns the actual number of primes between two numbers.
Used in the final worst witness cell/file. Doesn't use the Miller-Rabin primality test.
Credit: https://www.programiz.com/python-programming/examples/prime-number-intervals
'''

def actual_len_primes(lower, upper):
  for num in range(lower, upper + 1):
    if num > 1:
        for i in range(2, num):
            if (num % i) == 0:
                break
        else:
            return num

## Non-probabilistic primality test

In [4]:
'''
Returns whether a number is prime (true) or not (false).
Credit: https://djangocentral.com/python-program-to-check-whether-the-number-is-prime-or-not/
'''

def actual_prime(n):
  if n > 1:
    for i in range(2, int(n/2) + 1):
      if (n % i == 0):
        return True
      else:
        return False
  else:
    return False

## Finding worst witness in a certain range by finding difference with actual number of primes

In [17]:
# imports
from tqdm import tqdm

# constants
DEFENDANT = 30 # number we're using to test our function
WITNESS_RANGE = 30 # range till where we want to find the worst witness 
ACTUAL_LEN_PRIMES = actual_len_primes(0, WITNESS_RANGE)

# variables
results = {} # dictionary containing witness and their difference

# subtraction loop
for i in tqdm(range(WITNESS_RANGE)):
  results[i] = ACTUAL_LEN_PRIMES - len_probable_prime(DEFENDANT, i)

# finding the second worst key (worst witness within a range appears to always be 1)
print(list(results.keys())[list(results.values()).index(sorted(set(results.values()))[1])])

100%|██████████| 30/30 [00:00<00:00, 2315.84it/s]

7


## Finding worst witness for a single number by number of lies

# Other functions created during research


## Length of composites function

In [ ]:
'''
Returns the length of composite numbers of a given range.
'''

def len_composite(int, a):
    results = []

    for i in range(int + 1):
        if miller_rabin(i, a) == False:
            results.append(miller_rabin(i, a))

    return len(results)

## Composites function

In [ ]:
'''
Returns the composite numbers of a given range.
'''

def composite(int, a):
    results = []

    for i in range(int + 1):
        if miller_rabin(i, a) == False:
            results.append(i)

    return results

## Primes function

In [ ]:
'''
Returns the probable prime numbers of a given range.
'''

def probable_primes(int, a):
    results = []

    for i in range(int + 1):
        if miller_rabin(i, a) == True:
            results.append(i)

    return results

## Finding excellent, okay and false witnesses

In [ ]:
# imports
from tqdm import tqdm

# constants
DEFENDANT = 100
WITNESS_RANGE = 100
ACTUAL_LEN_PRIMES = actual_len_primes(0, WITNESS_RANGE)

# seconday variables 
excellent_witnesses = []  # = ACTUAL_LEN_PRIMES
okay_witnesses = []  # ±5 ACTUAL_LEN_PRIMES
false_witnessses = []  # != ACTUAL_LEN_PRIMES

# secondary loop
for i in tqdm(range(WITNESS_RANGE)):
    if len_probable_prime(DEFENDANT, i) == ACTUAL_LEN_PRIMES:
        excellent_witnesses.append(i)
    elif len_probable_prime(DEFENDANT, i) - ACTUAL_LEN_PRIMES <= 5:
        okay_witnesses.append(i)
    elif len_probable_prime(DEFENDANT, i) != ACTUAL_LEN_PRIMES:
        false_witnessses.append(i)

print(excellent_witnesses)

100%|██████████| 100/100 [00:00<00:00, 241.92it/s]

[]


## Finding worst witness by number of lies (similar to difference but doesn't appear to work)

In [31]:
# constants
DEFENDANT_RANGE = 10 # range of defendants we want to have heard
DEFENDANTS = [i for i in range(DEFENDANT_RANGE + 1)]
WITNESS_RANGE = 50 # range of witnesses we want to call

# variables
count = 0
index = DEFENDANT_RANGE - 1
results = {} # dictionary containing witness and number of lies

# liar loop
while index != -1:
  for i in range(WITNESS_RANGE):
    if miller_rabin(DEFENDANTS[index], i) != actual_prime(DEFENDANTS[index]):
      count += 1
      results[i] = count
  index -= 1

print(results)

# print(list(results.keys())[list(results.values()).index(sorted(set(results.values()))[0])])

{0: 301, 1: 302, 2: 303, 3: 304, 4: 305, 5: 306, 6: 307, 7: 308, 8: 309, 9: 310, 10: 311, 11: 312, 12: 313, 13: 314, 14: 315, 15: 316, 16: 317, 17: 318, 18: 319, 19: 320, 20: 321, 21: 322, 22: 323, 23: 324, 24: 325, 25: 326, 26: 327, 27: 328, 28: 329, 29: 330, 30: 331, 31: 332, 32: 333, 33: 334, 34: 335, 35: 336, 36: 337, 37: 338, 38: 339, 39: 340, 40: 341, 41: 342, 42: 343, 43: 344, 44: 345, 45: 346, 46: 347, 47: 348, 48: 349, 49: 350}
